Загалом, дескриптори - це об'єкти, для котрих перевизначена операція отримання об'єкта, задання об'єкта або видалення об'єкта.
Цей функціонал може бути перевантажений через методи \__get\__(), \__set\__() чи \__delete\__() відповідно.

Якщо в об'єкта визначений тільки метод \__get\__(), цей об'єкт називається non-data descriptor

In [66]:
class NotADescriptor:
    def __init__(self):
        print("not a descriptor")

class NonDataDescriptor:
    """Descriptor which defines only __get__ method named 'non-data descriptor'"""
    def __init__(self):
        print("descriptor")

    def __get__(self, inst, owner=None):
        print("__get__")
        return 5

class ExampleOne:
    d = NonDataDescriptor()
    e = NotADescriptor()
    f = 4
    

descriptor
not a descriptor


In [62]:
ExampleOne().e

In [63]:
ExampleOne().f

4

In [64]:
ExampleOne().d

__get__


5

Загалом, послідовність отримання атрибуту класу наступна:
* Python дивиться у data descriptors, що пов'язані з цим атрибутом
* Після цього Python дивиться у \__dict\__ об'єкта, де оголошений атрибут, котрий ми намагаємось отримати
* І, після цього, Python намагатиметься викликати non data descriptor для цього об'єкта

In [119]:
from typing import Any

class DataDescriptor:
    """Descriptor which defines not only __get__, but either __set__ or __delete__ methods is named 'data descriptor'"""
    def __init__(self):
        print("descriptor")

    def __get__(self, inst, owner=None) -> Any:
        print("__get__")
        return getattr(inst, "secret_name", 5)

    def __set__(self, inst, val) -> None:
        print(f"got __set__ for {inst} with {val}")
        inst.secret_name = val
              
    def __del__(self, inst) -> None:
        del inst.secret_name
        
class DataDescriptorWithSetName:
    def __set_name__(self, owner, name):
        print(self, owner, name)
        self.d_name = "_" + name
        
    def __get__(self, inst, owner=None) -> Any:
        print(self, inst, owner)
        print("__get__")
        return getattr(inst, self.d_name, 5)

    def __set__(self, inst, val) -> None:
        print(f"got __set__ for {inst} with {val}")
        setattr(inst, self.d_name, val)
              
    def __del__(self, inst) -> None:
        del inst.secret_name
        delattr(inst, self.d_name)
    

class ExampleTwo:
    a = NonDataDescriptor()
    b = DataDescriptor()
    c = DataDescriptorWithSetName()
    
    def __init__(self):
        self.__dict__["a"] = 100
        self.__dict__["b"] = 200


descriptor
descriptor
<__main__.DataDescriptorWithSetName object at 0x7fed782b5c90> <class '__main__.ExampleTwo'> c


In [100]:
ExampleTwo().__dict__["b"]

200

In [130]:
obj = ExampleTwo()
obj.c = 2

got __set__ for <__main__.ExampleTwo object at 0x7fed782b52a0> with 2


In [132]:
obj.__dict__

{'a': 100, 'b': 200, '_c': 2}

Більш приземлений приклад дескриптора: об'єкт, що видає розмір директорії

In [93]:
import os

class DirectorySize:

    def __get__(self, obj, objtype=None):
        #print(self, obj, objtype)
        #print(type(self), type(obj), type(objtype))
        return len(os.listdir(obj.dirname))
    
    def __repr__(self):
        return f"DirectorySizeObject"

class Directory:

    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname 
        
    def __repr__(self):
        return f"directory {self.dirname}"


In [94]:
Directory("/home/illia").size

102

In [33]:
vars(NotADescriptor)["__module__"]

'__main__'

Класи, що оголошують дескриптори - повноцінні. Тобто, вони можуть бути абстрактними, брати участь в ієрархії наслідування тощо. Ось приклад з офіційної документації Python з побудови валідатору за допомогою дескрипторів.

In [44]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        print(self, owner, name)
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass


In [45]:
class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

In [46]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

<__main__.String object at 0x7fed884a43d0> <class '__main__.Component'> name
<__main__.OneOf object at 0x7fed88493010> <class '__main__.Component'> kind
<__main__.Number object at 0x7fed88490760> <class '__main__.Component'> quantity


In [47]:
Component('Widget', 'metal', 5) 

ValueError: Expected <method 'isupper' of 'str' objects> to be true for 'Widget'

In [48]:
c = Component('WIDGET', 'metal', 5)

In [49]:
vars(c)

{'_name': 'WIDGET', '_kind': 'metal', '_quantity': 5}